In [1]:
import os 
import sys
import glob
import numpy as np
import itk
from itk import TubeTK as ttk
from itkwidgets import view

In [2]:
# NRRD Study Name
studyname = 'G:/My Drive/Projects/Proj_UNC_StrokeCollaterals/Experiments/UNC/CTP/CTAT-003'

# NRRD Files
directory = (studyname + '/')

# Saved NRRD Files 
directory2 = (studyname + '-Reg/')

# Mask Creation and Location
directory3 = (studyname + '-MinMax/')

pic_folder = os.listdir(directory)
pic_folder = [pic_folder for pic_folder in pic_folder if ".nii" in pic_folder]
pic_folder.sort()
print(pic_folder)
num_images = len(pic_folder)

im0Tmp = itk.imread(directory + pic_folder[int(num_images/2)], itk.F)

resample = ttk.ResampleImage.New(Input=im0Tmp,MakeIsotropic=True)
resample.Update()
im0 = resample.GetOutput()
immath = ttk.ImageMath.New(Input=im0)
immath.Blur(1)
im0Blur = immath.GetOutput()

immath.Threshold(150, 800, 1, 0)
immath.Dilate(10, 1, 0)
mask0 = immath.GetOutputUChar()
mask0Tmp = itk.GetArrayViewFromImage(mask0)
mask0Tmp[0:4,:,:] = 0
sizeZ = mask0Tmp.shape[0]
mask0Tmp[sizeZ-4:sizeZ,:,:] = 0   #No need to update mask0 since mask0Tmp is a view of mask0 (shared memory)

itk.imwrite(mask0, directory3 + 'mask.mha', compression=True)
maskObj = itk.ImageMaskSpatialObject[3].New()
maskObj.SetImage(mask0)
maskObj.Update()

['CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_10.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_11.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_12.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_13.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_14.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_15.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_16.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_17.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_18.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_19.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_20.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_21.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_22.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_23.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_24.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_25.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_26.nii', 'CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_27.nii', 'CTAT-003-Dawn_DAWN_PERFUSI

In [3]:
view(mask0)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [4]:
Dimension = 3
PixelType = itk.ctype('float')
ImageType = itk.Image[PixelType, Dimension]

imdatamax = itk.GetArrayFromImage(im0)
imdatamin = imdatamax
imdatamax2 = imdatamax
imdatamin2 = imdatamax
imdatamax3 = imdatamax
imdatamin3 = imdatamax

imFixedBlur = im0Blur

for imNum in range(num_images):
    imMoving = itk.imread( directory + pic_folder[imNum], itk.F )
    
    immath.SetInput(imMoving)
    immath.Blur(1)
    imMovingBlur = immath.GetOutput()
    
    imreg = ttk.RegisterImages[ImageType].New()
    imreg.SetFixedImage(imFixedBlur)
    imreg.SetMovingImage(imMovingBlur)
    
    imreg.SetRigidMaxIterations(3000)
    imreg.SetRegistration("RIGID")
    imreg.SetExpectedOffsetMagnitude(20)
    imreg.SetExpectedRotationMagnitude(0.3)
    imreg.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    
    imreg.SetFixedImageMaskObject(maskObj)
    #imreg.SetSampleFromOverlap(True)

    imreg.SetReportProgress(True)
    imreg.Update()
    
    tfm = imreg.GetCurrentMatrixTransform()
    #imFixedBlur = imreg.GetFinalMovingImage("LINEAR_INTERPOLATION", -1024)
    imMovingReg = imreg.ResampleImage("LINEAR_INTERPOLATION", imMoving, tfm, -1024)
    
    itk.imwrite( imMovingReg, directory2 + pic_folder[imNum], compression=True )
    
    print(tfm)
    
    imdataTmp = itk.GetArrayFromImage(imMovingReg)
    
    imdatamax = np.maximum(imdatamax,imdataTmp)
    imdatamin = np.minimum(imdatamin,imdataTmp)
    imdataTmp[np.where(imdataTmp==imdatamax)] = 0
    imdataTmp[np.where(imdataTmp==imdatamin)] = 0
    imdatamax2 = np.maximum(imdatamax2,imdataTmp)
    imdatamin2 = np.minimum(imdatamin2,imdataTmp)
    imdataTmp[np.where(imdataTmp==imdatamax)] = 0
    imdataTmp[np.where(imdataTmp==imdatamin)] = 0
    imdatamax3 = np.maximum(imdatamax3,imdataTmp)
    imdatamin3 = np.minimum(imdatamin3,imdataTmp)
    
    #out = itk.GetImageFromArray(imdatamax)
    #out.CopyInformation(im0)
    #itk.imwrite(out, (directory3 + 'max_' + str(imNum) + '.nrrd'))
    
    #out = itk.GetImageFromArray(imdatamax3)
    #out.CopyInformation(im0)
    #itk.imwrite(out, (directory3 + 'max3_' + str(imNum) + '.nrrd'))
    
    percent = (imNum + 1) / num_images * 100
    print(str(round(percent)) + '% : ' + pic_folder[imNum])
    
print('Done')    

AffineTransform (0000023224F3EB40)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 1462276
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999997 0.000123695 -0.0023051 
    -0.000127365 0.999999 -0.00159202 
    0.0023049 0.00159231 0.999996 
  Offset: [0.700482, 0.48526, 0.183581]
  Center: [0.203125, -194.703, 310.999]
  Translation: [-0.0404864, -0.00963405, -0.127199]
  Inverse: 
    0.999997 -0.000127365 0.0023049 
    0.000123695 0.999999 0.00159231 
    -0.0023051 -0.00159202 0.999996 
  Singular: 0

3% : CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_10.nii
AffineTransform (0000023224F3E760)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 2929748
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999999 -1.69432e-05 -0.00164264 
    1.588e-05 1 -0.000647238 
    0.00164265 0.000647211 0.999998 
  Offset: [0.470139, 0.183714, 0.186932]
  Center: [0.203

47% : CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_23.nii
AffineTransform (0000023224F3F300)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 21952452
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0.000138642 0.000193573 
    -0.000138444 0.999999 -0.00102422 
    -0.000193715 0.00102419 0.999999 
  Offset: [0.0012923, 0.343869, 0.112998]
  Center: [0.203125, -194.703, 310.999]
  Translation: [0.0344993, 0.0254143, -0.0866241]
  Inverse: 
    1 -0.000138444 -0.000193715 
    0.000138642 0.999999 0.00102419 
    0.000193573 -0.00102422 0.999999 
  Singular: 0

50% : CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_24.nii
AffineTransform (0000023224F3E950)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 23459450
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 -5.90988e-07 -4.14755e-06 
    5.91032e-07 1 1.06568e-05 
    4.14755e-06 -1.06568e-05 1 
  Offset: [0.

AffineTransform (0000023224F3ED30)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 41056675
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999994 -0.00056618 -0.00340493 
    0.000569993 0.999999 0.00111895 
    0.0034043 -0.00112088 0.999994 
  Offset: [0.838487, -0.534158, 0.267394]
  Center: [0.203125, -194.703, 310.999]
  Translation: [-0.110207, -0.185897, 0.484328]
  Inverse: 
    0.999994 0.000569993 0.0034043 
    -0.00056618 0.999999 -0.00112088 
    -0.00340493 0.00111895 0.999994 
  Singular: 0

93% : CTAT-003-Dawn_DAWN_PERFUSION_20180401112510_7.nii
AffineTransform (0000023224F3F110)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 42512401
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999998 0.000856008 -0.00156131 
    -0.000858991 0.999998 -0.00191123 
    0.00155967 0.00191257 0.999997 
  Offset: [0.56534, 0.540799, 0.429799]
  Center: 

In [5]:
out = itk.GetImageFromArray(imdatamax3)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'max3.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamin3)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'min3.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamax3 - imdatamin3)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'diff3.nrrd'), compression=True)

print('Done3')

Done3


In [6]:
out = itk.GetImageFromArray(imdatamax)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'max.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamin)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'min.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamax - imdatamin)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'diff.nrrd'), compression=True)

print('Done')

Done


In [7]:
out = itk.GetImageFromArray(imdatamax)
view(out)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…